# Linked Structures

## Agenda

1. Motivation
2. Wishlist
3. Mechanisms
4. Linked Data Structures

## 1. Motivation: built-in, array-backed list runtimes

What are the runtime complexities of some common operations for updating an array-backed list?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from timeit import timeit

### Index-based update

In [ ]:
size = 10_000

ts = [timeit('lst[{}] = None'.format(n),
             setup=f'lst = list(range({size}))',
             number=100)
      for n in range(0, size, 10)]

plt.plot(ts, 'ro');

### Appending

In [ ]:
ts = [timeit('lst.append(None)',
             setup=f'lst = list(range({n}))',
             number=10000)
      for n in np.linspace(1000, 100_000, 1000, dtype=int)]

plt.plot(ts, 'ro');

### Insertion & Deletion

In [ ]:
ns = np.linspace(100, 10000, 50, dtype=int)

# timing insertion at front
ins_ts = [timeit('lst.insert(0, None)',
                 setup=f'lst = list(range({n}))',
                 number=100)
          for n in ns]

# timing deletion at front
del_ts = [timeit('del lst[0]',
                 setup=f'lst = list(range({n}))',
                 number=100)
          for n in ns]

plt.plot(ns, ins_ts, 'go')
plt.plot(ns, del_ts, 'ro');

### Concatenation

In [ ]:
def concat1(lst1, lst2):
    for x in lst2:
        lst1.append(x)
    return lst1

def concat2(lst1, lst2):
    lst1.extend(lst2)
    return lst1

def concat3(lst1, lst2):
    return lst1 + lst2

In [ ]:
ns = np.linspace(100, 10000, 50, dtype=int)

ts = [[timeit(f'{fn}(lst1, lst2)',
              setup=f'lst1 = list(range({n})); lst2 = list(range({n}))',
              number=100,
              globals=globals())
       for n in ns] 
      for fn in ('concat1', 'concat2', 'concat3')]

plt.plot(ns, ts[0], 'ro')
plt.plot(ns, ts[1], 'go')
plt.plot(ns, ts[2], 'bo');

### Conclusion

While index-based updates and appends are $O(1)$, operations that may require changing the relative positions of elements en masse (e.g., insertion, deletion, concatenation) are $O(N)$. 

This is due to the **monolithic, contiguous memory allocation** of the array used to hold references to elements in the list.

## 2. Wishlist

For applications that frequently perform operations such as insertions & deletions (which are inefficient in an array-backed list) we would like to have an improved storage mechanism that:

- does **not** require monolithic, contiguous memory allocation,
- supports flexible and efficient reorganization of substructures (e.g., for concatenation and relocation),
- and allows us to build other commonly needed operations (e.g., for indexing, iteration, search) on top of it.

## 3. Mechanisms

### 3.1. Two-Element Lists

In [ ]:
# data items
i1 = 'lions'
i2 = 'tigers'
i3 = 'bears'
i4 = 'oh, my'

In [ ]:
# creating individual "links"
l1 = []
l2 = []
l3 = []
l4 = []

In [ ]:
# link-ing them together


In [ ]:
# data access
head = l1

In [ ]:
# iteration
def link_iterator(l):
    pass

In [ ]:
for x in link_iterator(head):
    print(x)

In [ ]:
# prepending
i0 = 'walruses'

In [ ]:
for x in link_iterator(head):
    print(x)

In [ ]:
# insertion
i2_5 = 'elephants'

In [ ]:
for x in link_iterator(head):
    print(x)

In [ ]:
# deletion


In [ ]:
for x in link_iterator(head):
    print(x)

### 3.2. "Node" objects

In [ ]:
class Node:
    def __init__(self, val, next=None):
        self.val = val
        self.next = next

In [ ]:
# manually constructing a list


In [ ]:
# data access


In [ ]:
# iteration
def node_iterator(l):
    pass

In [ ]:
for x in node_iterator(head):
    print(x)

## 4. Linked Data Structures

### 4.1 Linked List

In [ ]:
class LinkedList:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next = next

    def __init__(self):
        self.head = None
        
    def prepend(self, val):
        pass
        
    def __iter__(self):
        pass
            
    def __repr__(self):
        pass

In [ ]:
l = LinkedList()
for x in range(10):
    l.prepend(x)
l

### 4.2 Binary Tree

In [ ]:
class BinaryNode:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [ ]:
# an "expression tree" representing the arithmetic expression ((5+3)*(8-4))
t = BinaryNode('*')
t.left = BinaryNode('+')
t.left.left  = BinaryNode('5')
t.left.right = BinaryNode('3')
t.right = BinaryNode('-')
t.right.left  = BinaryNode('8')
t.right.right = BinaryNode('4')

In [ ]:
def print_expr_tree(t):
    if t:
        if not t.val.isdigit():
            print('(', end='')
        print_expr_tree(t.left)
        print(t.val, end='')
        print_expr_tree(t.right)
        if not t.val.isdigit():
            print(')', end='')

In [ ]:
print_expr_tree(t)

### 4.3 N-ary Tree

In [ ]:
class NaryNode:
    def __init__(self, val, n=2):
        self.val = val
        self.children = [None] * n
        
    def __getitem__(self, idx):
        return self.children[idx]
    
    def __setitem__(self, idx, val):
        self.children[idx] = val
        
    def __iter__(self):
        for c in self.children:
            yield c

In [ ]:
root = NaryNode('Kingdoms', n=5)

root[0] = NaryNode('Animalia', n=35)
root[1] = NaryNode('Plantae', n=12)
root[2] = NaryNode('Fungi', n=7)
root[3] = NaryNode('Protista', n=5)
root[4] = NaryNode('Monera', n=5)

root[2][0] = NaryNode('Chytridiomycota')
root[2][1] = NaryNode('Blastocladiomycota')
root[2][2] = NaryNode('Glomeromycota')
root[2][3] = NaryNode('Ascomycota')
root[2][4] = NaryNode('Basidiomycota')
root[2][5] = NaryNode('Microsporidia')
root[2][6] = NaryNode('Neocallimastigomycota')

def tree_iter(root):
    if root:
        yield root.val
        for c in root:
            yield from tree_iter(c)

In [ ]:
for x in tree_iter(root):
    print(x)